In [ ]:
import numpy as np
import h5py # You may need to install this in your python environment
import matplotlib.pyplot as plt

In [ ]:
# First, download the rgc_data from the website and save it in the same folder as this notebook
# Then run import the data
fpath = 'rgc_data.h5'
df = h5py.File(fpath, 'r')

In [ ]:
# Let's take a look at what data is in the file
list(df.keys())

In [ ]:
# Save the data to numpy arrays
spike_times = np.array(df['spike_times'])
stimulus = np.array(df['stimulus'])
time = np.array(df['time'])

In [ ]:
dt = 0.01

# Initialize the STE
# compute the dimensions of the filter (spatial and temporal)
spatial_dim = stimulus.shape[1] # the number of spatial dimensions in the stimulus
filter_length = 40 # the number of temporal dimensions in the stimulus

In [ ]:
# cut out the first few spikes that occur before the length of the filter (in seconds) has elapsed
spike_times = spike_times[np.where(spike_times > filter_length*dt)[0]]

In [ ]:
# store the indices of the time array corresponding to each spike
# (you'll use this when computing histograms and the nonlinearity of the LN model)
spike_indices = np.zeros(len(spike_times))
time_indices_of_spikes = np.zeros(len(spike_times))

# Q1.2
STES = np.zeros((len(spike_times), filter_length*spatial_dim))

# Q1.3 Collect stimuli that are part of the STE
# select out the stimulus preceeding the spike, and store it in the `ste` array
for i, st in enumerate(spike_times):
    
    # Your code here.
    # A possible answer. Please comment the code to explain what is happening.
    time_diffs = np.abs(time-st)
    time_index = np.argmin(time_diffs)
    time_indices_of_spikes[i] = time_index
    
    spike_indices[i] = time_index
    
    starting_stim_ind = time_index-filter_length
    current_ste = stimulus[starting_stim_ind:time_index, :]
    reshaped_ste = np.reshape(current_ste, (1, 1200))
    
    STES[i] = reshaped_ste

In [ ]:
# Q1.4

# Compute the spike-triggered average (STA)
# by averaging over the number of spikes in the STE

# Take the mean of the STE data in the correct axis
STA_vector = np.mean(STES, 0)

# Normalize the vector such that is has a unit norm
STA_vector_normalized = STA_vector/np.linalg.norm(STA_vector)

# Reshape the data for plotting
STA = np.reshape(STA_vector_normalized, (filter_length, spatial_dim))

# For plotting: this colormap works well, but you can use whichever you want
colormap = 'RdBu_r'

plt.figure()
plt.imshow(np.flip(STA, 0), origin='lower', cmap=colormap)

# Depending on how you plotted the data, you may have to switch the x and y labels
plt.ylabel("Time before spike, tens of milliseconds")
plt.xlabel("Pixel of stimulus space")

plt.show()

In [ ]:
# Spike-triggered covariance
ste_cov = np.cov(STES, rowvar = False)
ste_cov.shape

# Eigenvectors
eigenvals, eigenvecs = np.linalg.eig(ste_cov)

In [ ]:
# Your code here. An example with the first eigenvector is plotted below.
# Copy and paste this code and change the eig_index to plot other eigenvectors
# Add axes labels
eig_index = 0
eigen_stc_to_plot = np.reshape(eigenvecs[:, eig_index], (filter_length, spatial_dim))

plt.figure()
plt.imshow(np.flip(eigen_stc_to_plot), origin='lower', cmap = colormap)

#plt.xlabel(/*Your code here*/)
#plt.ylabel(/*Your code here*/)
plt.title("First Eigenvector of Spike-triggered Covariance")
plt.show()

In [ ]:
# Your code here for E-H

In [ ]:
plt.figure()
plt.plot(eigenvals, '-', markersize = 3, color = "xkcd:green")

#plt.xlabel(/*Your code here*/)
#plt.ylabel(/*Your code here*/)
plt.title("Eigenspectrum of covariance matrix")

plt.show()

In [ ]:
# Linear non-linear analysis
# This is functioning example code, look through and adjust as necessary

In [ ]:
def custom_histogram(x, nbins):
    
    edges = np.linspace(min(x), max(x), nbins)
    n, edges = np.histogram(x, edges)
    inds = np.digitize(x, edges)
    
    return n, edges, inds

In [ ]:
# LN modeling
u = np.zeros((len(time)))

for t in range(filter_length, len(time)):
    
    stimulus_slice = stimulus[t-filter_length:t, :]
    stim_slice_flat = np.reshape(stimulus_slice, (1, filter_length*spatial_dim))
    u[t] = np.dot(stim_slice_flat, STA_vector_normalized)

In [ ]:
n, edges, bin_indices = custom_histogram(u, 50)
spike_counts = np.histogram(spike_times, np.append(time, [time[-1]+dt]))
ub = np.unique(bin_indices)

nonlinearity = np.zeros((len(ub)))
bin_centers = edges[0:-1] + 0.5*np.diff(edges)

for bin_index in range(len(ub)):
        
    # find the times where u is in bin_index
    inds_u_in_bin = np.where(bin_indices == ub[bin_index])[0]
    mean_spike_count = np.mean(spike_counts[0][inds_u_in_bin])
    
    nonlinearity[bin_index] = mean_spike_count

In [ ]:
plt.figure(figsize = (5, 5))
plt.plot(bin_centers, nonlinearity, '-o', markersize = 3, color = 'xkcd:medium pink')

#plt.xlabel(/*Your code here*/)
#plt.ylabel(/*Your code here*/)
#plt.title(/*Your code here*/)
plt.show()